# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our objective function. 
After training, we will need these parameters 
in order to make future predictions.
Additionally, we will sometimes wish 
to extract the parameters 
either to reuse them in some other context,
to save our model to disk so that 
it may be executed in other software,
or for examination in the hopes of 
gaining scientific understanding.

Most of the time, we will be able 
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on the framework
to do the heavy lifting.
However, when we move away from 
stacked architectures with standard layers, 
we will sometimes need to get into the weeds
of declaring and manipulating parameters. 
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Parameter initialization.
* Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
x = torch.randn(2, 4)
net(x)

tensor([[-0.4553],
        [-0.5533]], grad_fn=<AddmmBackward>)

## Parameter Access

Let us start with how to access parameters
from the models that you already know.
When a model is defined via the Sequential class,
we can first access any layer by indexing 
into the model as though it were a list.
Each layer's parameters are conveniently 
located in its attribute. 
We can inspect the parameters of the `net` defined above as a dictionary.

In [2]:
print(net[2].state_dict())  

OrderedDict([('weight', tensor([[-0.2824, -0.3416,  0.1449, -0.1888,  0.3456, -0.0462, -0.1817, -0.2644]])), ('bias', tensor([-0.2644]))])


The output tells us a few important things.
First, each fully-connected layer 
contains two parameters, e.g., 
`weight` and `bias` (may with prefix),
corresponding to that layer's 
weights and biases, respectively.
Both are stored as single precision floats.
Note that the names of the parameters
allow us to *uniquely* identify
each layer's parameters,
even in a network containing hundreds of layers.


### Targeted Parameters

Note that each parameter is represented
as an instance of the `Parameter` class.
To do anything useful with the parameters,
we first need to access the underlying numerical values. 
There are several ways to do this.
Some are simpler while others are more general.
To begin, given a layer, 
we can access one of its parameters 
via the `bias` or `weight` attributes, which returns an `Parameter` instance
and further access that parameter's value
via its `data` method.
The following code extracts the bias
from the second neural network layer.

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2644], requires_grad=True)
tensor([-0.2644])


Parameters are complex objects,
containing data, gradients,
and additional information.
That's why we need to request the data explicitly.


In addition to `data`, each `Parameter` also provides a `grad` method for accessing the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

In [4]:
net[0].weight.grad == None

True

### All Parameters at Once

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex blocks, (e.g., nested Blocks),
since we would need to recurse 


through the entire tree in to extract
each sub-block's parameters.

In [5]:
# parameters only for the first layer
print(net[0].state_dict())
# parameters of the entire network
print(net.state_dict())

OrderedDict([('weight', tensor([[ 0.0473,  0.0981,  0.3233,  0.0945],
        [ 0.4139, -0.3490, -0.4519, -0.1895],
        [-0.0101,  0.1232, -0.4548,  0.0163],
        [-0.4418,  0.2759,  0.2210, -0.2635],
        [-0.2618,  0.4638,  0.2510, -0.4024],
        [-0.0405, -0.1478, -0.1023, -0.0862],
        [ 0.1349, -0.2153, -0.1138,  0.3552],
        [-0.3953,  0.1928, -0.1807, -0.0294]])), ('bias', tensor([-0.2545,  0.3415, -0.3311, -0.2528,  0.2650,  0.1262, -0.4438,  0.4875]))])
OrderedDict([('0.weight', tensor([[ 0.0473,  0.0981,  0.3233,  0.0945],
        [ 0.4139, -0.3490, -0.4519, -0.1895],
        [-0.0101,  0.1232, -0.4548,  0.0163],
        [-0.4418,  0.2759,  0.2210, -0.2635],
        [-0.2618,  0.4638,  0.2510, -0.4024],
        [-0.0405, -0.1478, -0.1023, -0.0862],
        [ 0.1349, -0.2153, -0.1138,  0.3552],
        [-0.3953,  0.1928, -0.1807, -0.0294]])), ('0.bias', tensor([-0.2545,  0.3415, -0.3311, -0.2528,  0.2650,  0.1262, -0.4438,  0.4875])), ('2.weight', tensor([

This provides us with another way of accessing the parameters of the network:

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.2644])

### Collecting Parameters from Nested Blocks

Let us see how the parameter naming conventions work 
if we nest multiple blocks inside each other. 
For that we first define a function that produces blocks 
(a block factory, so to speak) and then 
combine these inside yet larger blocks.

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[-0.0840],
        [-0.0836]], grad_fn=<AddmmBackward>)

Now that we have designed the network, 
let us see how it is organized.

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


Since the layers are hierarchically nested,
we can also access them as though 
indexing through nested lists. 
For instance, we can access the first major block, 
within it the second subblock, 
and within that the bias of the first layer,
with as follows:

In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.2157, -0.2301,  0.2902,  0.4247, -0.0564, -0.0768,  0.2091, -0.3472])

## Parameter Initialization

Now that we know how to access the parameters,
let us look at how to initialize them properly.
We discussed the need for initialization in :numref:`sec_numerical_stability`. 


By default, PyTorch initializes weight and bias matrices
uniformly by drawing from a range that is computed according to the input and output dimension. 
However, we will often want to initialize our weights
according to various other protocols. 
PyTorch's `nn.init` module provides a variety 
of preset initialization methods.
If we want to create a custom initializer,
we need to do some extra work.


### Built-in Initialization

Let us begin by calling on built-in initializers. 
The code below initializes all weight parameters 
as Gaussian random variables 
with standard deviation $.01$, while bias parameters set to 0.

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(net[0].weight, mean=0, std=0.01)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)    
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0135,  0.0106, -0.0094,  0.0029]), tensor(0.))

We can also initialize all parameters 
to a given constant value (say, $1$).

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(net[0].weight, 1)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)    
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

We can also apply different initializers for certain Blocks.
For example, below we initialize the first layer
with the `Xavier` initializer
and initialize the second layer 
to a constant value of 42.

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6864, -0.1805, -0.0108,  0.4538])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### Custom Initialization

Sometimes, the initialization methods we need 
are not provided in the `init` module. 
In the example below, we define an initializer
for the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Again, we implement a `my_init` function to apply to `net`.

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5    
    
net.apply(my_init)
net[0].weight[0:2]

tensor([[-8.2880, -0.0000, -0.0000, -0.0000],
        [-0.0000,  0.0000,  9.3321, -9.1667]], grad_fn=<SliceBackward>)

Note that we always have the option 
of setting parameters directly by calling `data` 
to access the underlying data. 


In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

## Tied Parameters

Often, we want to share parameters across multiple layers.
Later we will see that when learning word embeddings,
it might be sensible to use the same parameters
both for encoding and decoding words. 
We discussed one such case when we introduced :numref:`sec_model_construction`. 
Let us see how to do this a bit more elegantly. 
In the following we allocate a dense layer 
and then use its parameters specifically 
to set those of another layer.

In [15]:
# We need to give the shared layer a name such that we can reference its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))
net(x)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters 
of the second and third layer are tied. 
They are not just equal, they are 
represented by the same exact tensor. 
Thus, if we change one of the parameters,
the other one changes, too. 
You might wonder, 
*when parameters are tied
what happens to the gradients?*
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.


## Exercises

1. Use the FancyMLP defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the `init` module document to explore different initializers.
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


[Discussions](https://discuss.d2l.ai/t/57)